In [ ]:
# Cold spray powder feeder flow rate tracker V1.0

import tkinter as tk
import sys
from tkinter import filedialog
from tkinter import ttk
import customtkinter 
from mttkinter import mtTkinter as tk
from tkinter.font import Font
import io
import glob
import csv
import threading
from threading import Thread
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib
matplotlib.use("TkAgg")
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import xlsxwriter
import time
import os
import xlwings as xw
import random

################# Simulate incoming data #################################

def current_time_stream():
    while True:
        yield time.time()
        time.sleep(1/3)

def random_values_stream(start, end):
    current_value = start
    while current_value >= end:
        yield current_value
        current_value -= random.uniform(-0.0001, 0.0003)
        time.sleep(1/9)

# Create generators for the three data streams
time_stream = current_time_stream()
random_stream_1 = random_values_stream(3, 0)
random_stream_2 = random_values_stream(4, 0)


##################initializations###############################



mov_avg_array = np.empty((0, 2))
time_array = np.array([]) 
output_array=np.array([])
output_time_array=np.array([])
data_all = []
time_all = []
data_matrix = np.empty((0, 3)) 
file_path4 = None
data_lock = threading.Lock()
should_run = False
scatter_plot = None
ax = None

##################calculation functions###############################


def prepareplots():
    
    global ax
    
    if ax != None:

        ax.clear()    
        ax.set_xlabel('X-axis')
        ax.set_ylabel('Y-axis')
        ax.set_title('Scatter Plot Example')
    else:
        pass


def convert_excel_serial_to_datetime(serial_value):

    excel_epoch = datetime(1899, 12, 30)  # Excel epoch starts from December 30, 1899
    delta_days = timedelta(days=int(serial_value))
    delta_time = timedelta(seconds=(serial_value % 1) * 86400)  # Convert decimal part to seconds
    result_datetime = excel_epoch + delta_days + delta_time
    return result_datetime

def add_data_to_matrix(data, time_stamp):
    global data_matrix
    global PF1
    global PF2
    global window_size_mvg_avg
    global window_size_pfr_calc
    global valid_mvg_avg_change
    global window_size_sec_fil
    global std_deviation_threshold_sec_fil
    
    window_size_sec_fil= int(slider2aab.get())
    std_deviation_threshold_sec_fil = slider2aac.get()
    
    window_size_mvg_avg = int(slider2e.get())
    window_size_pfr_calc = int(slider2g.get())
    valid_mvg_avg_change = slider2f.get()

      
    PF1_var = selected_var1.get()
    PF2_var = selected_var2.get()
    
    if PF1_var:
        PF1 = 'ON'
    else:
        PF1 = 'OFF'
        
    if PF2_var:
        PF2 = 'ON'
    else:
        PF2 = 'OFF'

    data_all.append(data) 
    time_all.append(time_stamp)

    df_all_scales = pd.DataFrame(data_all)
    df_all_scales['Time'] = time_all

    
    if PF1 =='ON' and PF2 =='ON':
        waiting_label.config(text="Waiting for data...")
        # first filter raw data
        if len(data_matrix) ==0:

            data_matrix = np.vstack((data_matrix, data))
            waiting_label2.config(text="Waiting for data from powder feeder 1 & 2")
            waiting_label.place_forget()
        elif len(data_matrix)!=0 and len(data_matrix) < window_size_sec_fil and ((data_matrix[-1][1] - data[1]) >=0) and((data_matrix[-1][2] - data[2]) >=0) and data[1] >0 and data[2] >0 and (data_matrix[-1][0]!= data[0]):

            data_matrix = np.vstack((data_matrix, data))
            waiting_label2.config(text="Waiting for data from powder feeder 1 & 2")
            waiting_label.place_forget()
        # second filter raw data
        elif len(data_matrix) >= window_size_sec_fil and ((data_matrix[-1][1] - data[1]) >=0) and((data_matrix[-1][2] - data[2]) >=0) and data[1] >0 and data[2] >0 and (data_matrix[-1][0]!= data[0]):

            last_n_rows = data_matrix[-window_size_sec_fil:]
            sum_rowwise = np.sum(last_n_rows[:, 1:3], axis=1)
            avg = np.mean(sum_rowwise)
            std = np.std(sum_rowwise)
            current_value = data[1]+data[2]
            difference = abs(current_value - avg)
            max_std = std_deviation_threshold_sec_fil * std

            if difference > max_std:
                pass
            else:
                data_matrix = np.vstack((data_matrix, data))

        else:
            waiting_label2.config(text="Waiting for data from powder feeder 1 & 2")
            waiting_label.place_forget()

    elif PF1 =='ON' and PF2 =='OFF':
        waiting_label.config(text="Waiting for data...")

        if len(data_matrix) ==0:
            data_matrix = np.vstack((data_matrix, data))
            waiting_label2.config(text="Waiting for data from powder feeder 1")
            waiting_label.place_forget()
        elif len(data_matrix)!=0 and len(data_matrix) < window_size_sec_fil and ((data_matrix[-1][1] - data[1]) >=0) and data[1] >0 and (data_matrix[-1][0]!= data[0]):
            data_matrix = np.vstack((data_matrix, data))
            waiting_label2.config(text="Waiting for data from powder feeder 1")
            waiting_label.place_forget()
        # second filter raw data
        elif len(data_matrix) >= window_size_sec_fil and ((data_matrix[-1][1] - data[1]) >=0) and data[1] >0 and (data_matrix[-1][0]!= data[0]):
            last_n_rows = data_matrix[-window_size_sec_fil:]
            sum_rowwise = np.sum(last_n_rows[:, 1:2], axis=1)
            avg = np.mean(sum_rowwise)
            std = np.std(sum_rowwise)
            current_value = data[1]
            difference = abs(current_value - avg)
            max_std = std_deviation_threshold_sec_fil * std
            
            if difference > max_std:
                pass
            else:
                data_matrix = np.vstack((data_matrix, data))

        else:
            waiting_label2.config(text="Waiting for data from powder feeder 1")
            waiting_label.place_forget()

    elif PF1 =='OFF' and PF2 =='ON':
        waiting_label.config(text="Waiting for data...")

        # first filter raw data
        if len(data_matrix) ==0:
            data_matrix = np.vstack((data_matrix, data))
            waiting_label2.config(text="Waiting for data from powder feeder 2")
            waiting_label.place_forget()
        elif len(data_matrix)!=0 and len(data_matrix) < window_size_sec_fil and((data_matrix[-1][2] - data[2]) >=0) and data[2] >0 and (data_matrix[-1][0]!= data[0]):
            data_matrix = np.vstack((data_matrix, data))
            waiting_label2.config(text="Waiting for data from powder feeder 2")
            waiting_label.place_forget()
        # second filter raw data
        elif len(data_matrix) >= window_size_sec_fil and((data_matrix[-1][2] - data[2]) >=0) and data[2] >0 and (data_matrix[-1][0]!= data[0]):
            last_n_rows = data_matrix[-window_size_sec_fil:]
            sum_rowwise = np.sum(last_n_rows[:, 2:3], axis=1)
            avg = np.mean(sum_rowwise)
            std = np.std(sum_rowwise)
            current_value = data[2]
            difference = abs(current_value - avg)
            max_std = std_deviation_threshold_sec_fil * std
            
            if difference > max_std:
                pass
            else:
                data_matrix = np.vstack((data_matrix, data))
        else:
            waiting_label2.config(text="Waiting for data from powder feeder 2")
            waiting_label.place_forget()

    elif PF1 == "OFF" and PF2 =='OFF':
        # Hide the label if the condition is met
        waiting_label.config(text="Select Powder feeder")
        waiting_label2.place_forget()

def analyze_matrix(matrix):
    
    global mov_avg_array
    global output_array
    global output_time_array
    global scatter_plot
    global ax
    global PF1
    global PF2
    global window_size_mvg_avg
    global window_size_pfr_calc
    global valid_mvg_avg_change
    
    if (len(matrix) > window_size_mvg_avg-1):
        matrix2 = np.array(matrix)

        if PF1 =='ON' and PF2 =='ON':

            moving_average1 = np.mean(matrix2[-window_size_mvg_avg:, 1])
            moving_average2 = np.mean(matrix2[-window_size_mvg_avg:, 2])

            moving_average = moving_average1 + moving_average2
            
            time = matrix2[-1:, 0][0]
            
        elif PF1 =='ON' and PF2 =='OFF':
            moving_average = np.mean(matrix2[-window_size_mvg_avg:, 1])
            
            time = matrix2[-1:, 0][0]
            
        elif PF1 =='OFF' and PF2 =='ON':
            moving_average = np.mean(matrix2[-window_size_mvg_avg:, 2])
            
            time = matrix2[-1:, 0][0]

        elif PF1 == "all" and PF2 =='all':
            moving_average1 = np.mean(matrix2[-window_size_mvg_avg:, 1])
            moving_average2 = np.mean(matrix2[-window_size_mvg_avg:, 2])

            moving_average = moving_average1 + moving_average2
            
            time = matrix2[-1:, 0][0]
            

        # Check if there are already values in the the moving average matrix (mov_avg_array)
        if len(mov_avg_array)> 0:
            
            # calculate difference between current and previously calculated moving average
            difference_calculated = abs(mov_avg_array[-1][1]-moving_average)
            
            # if the difference between between current and previously calculated moving average is bigger than 0.2 consider the value as an error and do not include it
            if difference_calculated > valid_mvg_avg_change:
                pass
            
            # if the difference between between current and previously calculated moving average is smaller than 0.2, include it
            else:

                list_mov_avg_and_time=[]
                list_mov_avg_and_time=[time,moving_average]
                mov_avg_array = np.vstack((mov_avg_array,list_mov_avg_and_time))
                
                if (mov_avg_array[-1][0]) == (mov_avg_array[-2][0]):
                    mov_avg_array = mov_avg_array[:-1, :]
                else:
                    pass
                                       
            #Calculating PF flow rate 
            if len(mov_avg_array)> window_size_pfr_calc:
                last_N_rows = mov_avg_array[-window_size_pfr_calc:]
                val_1 = last_N_rows[0,1]
                val_1_time = last_N_rows[0,0]
                val_2 = last_N_rows[-1,1]
                val_2_time = last_N_rows[-1,0]
                mov_avg_diff = val_1 - val_2
                time_difference = abs(val_1_time-val_2_time)

                if time_difference > 0:
                    if time_difference > 10000:
                        # to account for time jump at midnight 86400 -> 00000
                        pass
                    else:
                        result = ((val_1 - val_2) / time_difference)*3600
                        output_array  = np.append(output_array,result)
                        output_time_array  = np.append(output_time_array,val_2_time)
                        
                        if len(output_time_array) < 2:
                            pass
                        else:
                            if (output_time_array[-1]) == (output_time_array[-2]):
                                output_time_array = output_time_array[:-1]
                                output_array = output_array[:-1]
                            else:
                                pass
                else:
                    pass
            else:
                pass

            if ax is None:
                ax = fig.add_subplot(111)
        
            # Generate some sample data for demonstration
            x_data = output_time_array
            y_data = output_array

            if y_data.size>0:
            # Hide the label if the condition is met
                waiting_label.place_forget()
                waiting_label2.place_forget()
            else:
                # Show the label if the condition is not met
                waiting_label.place(x=400, y=200)

            prepareplots()
            scatter_plot = ax.scatter(x_data, y_data)
            ax.set_xlabel('Time[s]')
            ax.set_ylabel('Powder flow rate [kg/hr]')
            ax.set_title('Simulated powder flow rate')
            canvas.draw()
        
        elif len(mov_avg_array)== 0:

            list_mov_avg_and_time=[]
            list_mov_avg_and_time=[time,moving_average]
            mov_avg_array = np.vstack((mov_avg_array,list_mov_avg_and_time))
    else:
        pass


def convert_excel_serial_to_datetime(serial_value):
    excel_epoch = datetime(1899, 12, 30)  # Excel epoch starts from December 30, 1899
    delta_days = timedelta(days=int(serial_value))
    delta_time = timedelta(seconds=(serial_value % 1) * 86400)  # Convert decimal part to seconds
    result_datetime = excel_epoch + delta_days + delta_time
    return result_datetime

def time_to_seconds(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    total_seconds = (hours * 3600) + (minutes * 60) + seconds
    return total_seconds
    
def read_excel_data():
    global should_run
    with data_lock:
        while should_run == True:
            times = next(time_stream)
            r1 = next(random_stream_1)
            r2 = next(random_stream_2)
            formatted_time = datetime.fromtimestamp(times).strftime("%H:%M:%S")
            total_seconds = time_to_seconds(formatted_time)
            data_array = np.array([total_seconds, r1, r2])
            add_data_to_matrix(data_array,formatted_time)
            analyze_matrix(data_matrix)
            time.sleep(0.1)

##################Tkinter widgets functions###############################


def save_csv_file():
    global data_all
    global time_all
    global mov_avg_array
    global mov_avg_array2
    global output_array
    global output_time_array

    df_all_scales = pd.DataFrame(data_all, columns=['CH0', 'CH1', 'CH2'])
    # Rename the specific columns using the rename method
    df_all_scales['TIME'] = time_all
    file_path3 = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV Files", "*.csv")])
    if file_path3:
        # Save the DataFrame to a CSV file
        df_all_scales.to_csv(file_path3, index=False)
    
def slider_event2aab(value):
    value2aab = slider2aab.get()
    value2aab= round(value2aab, 0)
    output_value2aab.set(value2aab)
    
def slider_event2aac(value):
    value2aac = slider2aac.get()
    value2aac= round(value2aac, 1)
    output_value2aac.set(value2aac)
      
def slider_event2e(value):
    value2e = slider2e.get()
    value2e= round(value2e, 0)
    output_value2e.set(value2e)
    
def slider_event2f(value):
    value2f = slider2f.get()
    value2f= round(value2f, 1)
    output_value2f.set(value2f)
    
def slider_event2g(value):
    value2g = slider2g.get()
    value2g= round(value2g, 0)
    output_value2g.set(value2g)

def start_reading():
    global should_run
    should_run = True
    t1 = threading.Thread(target=read_excel_data,daemon=True)
    t1.start()
    browse_button3["state"] = "disabled"
    slider2aab.configure(state="disabled",button_color='gray88')
    slider2aac.configure(state="disabled",button_color='gray88')
    slider2e.configure(state="disabled",button_color='gray88')
    slider2f.configure(state="disabled",button_color='gray88')
    slider2g.configure(state="disabled",button_color='gray88')
    checkbox1["state"] = "disabled"
    checkbox2["state"] = "disabled"
    button1["state"] = "disabled"
    button3["state"] = "disabled"

        
        

def stop_reading():
    global should_run
    should_run = False
    browse_button3["state"] = "normal"
    slider2aab.configure(state="normal",button_color='DodgerBlue3')
    slider2aac.configure(state="normal",button_color='DodgerBlue3')
    slider2e.configure(state="normal",button_color='DodgerBlue3')
    slider2f.configure(state="normal",button_color='DodgerBlue3')
    slider2g.configure(state="normal",button_color='DodgerBlue3')
    checkbox1["state"] = "normal"
    checkbox2["state"] = "normal"
    button1["state"] = "normal"
    button3["state"] = "normal"


def button3_event():
    global mov_avg_array  
    global time_array 
    global output_array
    global output_time_array
    global data_all 
    global time_all 
    global data_matrix 
                                       
    mov_avg_array = np.empty((0, 2))  
    time_array = np.array([]) 
    output_array=np.array([])
    output_time_array=np.array([])
    data_all = []
    time_all = []
    data_matrix = np.empty((0, 3))

    prepareplots()
    
    canvas.draw()
    waiting_label.config(text="Waiting for data...")
    waiting_label.place(x=400, y=200)

##################Tkinter widgets elements###############################    

root = tk.Tk()
root.title("Powder flow rate tracker")
root.geometry('1000x1000')
root.resizable(False, False)  # Disable window resizing

# Create a canvas
canvas = tk.Canvas(root, width=1000, height=1000)
canvas.place(x=0, y=0)

# Create a larger frame inside the canvas
main_frame = tk.Frame(canvas, width=1000, height=1200)
main_frame.place(x=0, y=0)


######################### First frame ###################################################################

first_frame = tk.Frame(main_frame, width=900, height=60, highlightbackground='black', highlightthickness=1)
first_frame.place(x=50, y=10)

label1a = tk.Label(first_frame, text="Powder flow rate tracker", font=("Arial", 15, "bold"))
label1a.place(x=280, y=20)


######################### Second frame ###################################################################

second_frame = tk.Frame(main_frame, width=900, height=470, highlightbackground='black', highlightthickness=1)
second_frame.place(x=50, y=100)

########################### Step 1

label2a = tk.Label(second_frame, text="Step 1: No connection to data logger required. This program uses simulated powder feeder data.")
label2a.place(x=10, y=20)

########################### Step 2

label2aa = tk.Label(second_frame, text="Step 2: Filter/Calculation settings:")
label2aa.place(x=10, y=60)

custom_font = Font(family="Helvetica", size=10, underline=True)

################ Raw data filter

label2aaa = ttk.Label(second_frame, text="Raw data filter", font=custom_font)
label2aaa.place(x=40, y=100)

####-----------Window size

label2aab = ttk.Label(second_frame, text="Window size")
label2aab.place(x=40, y=130)

slider2aab = customtkinter.CTkSlider(master=second_frame, from_=5, to=20,command=slider_event2aab, state="normal", button_color='DodgerBlue3', width= 70)
slider2aab.place(x=190, y=133)
slider2aab.set(5)

output_value2aab = tk.StringVar() # Create StringVar() variable
output_value2aab.set('5')
output_field2aab = tk.Entry(second_frame, textvariable=output_value2aab, state='readonly', width = 5)
output_field2aab.place(x=270,y=133)

####----------Standard deviation factor

label2aac = ttk.Label(second_frame, text="Standard deviation factor")
label2aac.place(x=40, y=160)

slider2aac = customtkinter.CTkSlider(master=second_frame, from_=0.1, to=5,command=slider_event2aac, state="normal", button_color='DodgerBlue3', width= 70)
slider2aac.place(x=190, y=163)
slider2aac.set(4.5)

output_value2aac = tk.StringVar() # Create StringVar() variable
output_value2aac.set('4.5')
output_field2aac = tk.Entry(second_frame, textvariable=output_value2aac, state='readonly', width = 5)
output_field2aac.place(x=270,y=163)

################ Moving average

label2aad = ttk.Label(second_frame, text="Moving average calculation", font=custom_font)
label2aad.place(x=340, y=100)

####-----------Window size

label2e = tk.Label(second_frame, text="Window size")
label2e.place(x=340, y=130)

slider2e = customtkinter.CTkSlider(master=second_frame, from_=5, to=20,command=slider_event2e, state="normal", button_color='DodgerBlue3', width= 70)
slider2e.place(x=490, y=133)
slider2e.set(5)

output_value2e = tk.StringVar() # Create StringVar() variable
output_value2e.set('5')
output_field2e = tk.Entry(second_frame, textvariable=output_value2e, state='readonly', width = 5)
output_field2e.place(x=570,y=133)

####-----------Max allowed change

label2f = tk.Label(second_frame, text="Max allowed change")
label2f.place(x=340, y=160)

slider2f = customtkinter.CTkSlider(master=second_frame, from_=0, to=5,command=slider_event2f, state="normal", button_color='DodgerBlue3', width= 70)
slider2f.place(x=490, y=163)
slider2f.set(0.2)

output_value2f = tk.StringVar() # Create StringVar() variable
output_value2f.set('0.2')
output_field2f = tk.Entry(second_frame, textvariable=output_value2f, state='readonly', width = 5)
output_field2f.place(x=570,y=163)


################ Powder flow rate calculation

label2aad = ttk.Label(second_frame, text="Powder flow calculation", font=custom_font)
label2aad.place(x=640, y=100)

############### Window Powder feeder rate

label2g = tk.Label(second_frame, text="Window size")
label2g.place(x=640, y=130)

slider2g = customtkinter.CTkSlider(master=second_frame, from_=5, to=300,command=slider_event2g, state="normal", button_color='DodgerBlue3', width= 70)
slider2g.place(x=730, y=133)
slider2g.set(5)

output_value2g = tk.StringVar() # Create StringVar() variable
output_value2g.set('5')
output_field2g = tk.Entry(second_frame, textvariable=output_value2g, state='readonly', width = 5)
output_field2g.place(x=810,y=133)


########################### Step 3

label2c = tk.Label(second_frame, text="Step 3: Select powder feeder in use:")
label2c.place(x=10, y=210)


selected_var1 = tk.IntVar()
# Create a Checkbutton
checkbox1 = tk.Checkbutton(second_frame, text="Powder feeder 1 (Scale 1)", variable=selected_var1)
checkbox1.place(x=40, y=240)



# Create a variable to hold the checkbox state
selected_var2 = tk.IntVar()
# Create a Checkbutton
checkbox2 = tk.Checkbutton(second_frame, text="Powder feeder 2 (Scale 2)", variable=selected_var2)
checkbox2.place(x=40, y=260)


########################### Step 4


label2h = tk.Label(second_frame, text="Step 4: Live tracking of Powder flow rate [kg/hr]:")
label2h.place(x=10, y=310)

button1 = tk.Button(second_frame, text="Start", width=5, height=1,command=start_reading)
button1.place(x=40, y=340)

button2 = tk.Button(second_frame, text="Stop", width=5, height=1,command=stop_reading)
button2.place(x=100, y=340)

button3 = tk.Button(second_frame, text="Reset", width=5, height=1,command=button3_event)
button3.place(x=160, y=340)

########################### Step 5

label2i = tk.Label(second_frame, text="Step 5: Export raw data as csv file:")
label2i.place(x=10, y=390)

browse_button3 = tk.Button(second_frame, text="Save", command=save_csv_file, width= 15)
browse_button3.place(x=40, y=420)


######################### third frame ###################################################################

third_frame = tk.Frame(main_frame, width=900, height=330, highlightbackground='black', highlightthickness=1)
third_frame.place(x=50, y=600)

fig = plt.Figure(figsize=(5, 3), dpi=100, tight_layout=True)
# Create a FigureCanvasTkAgg widget
canvas = FigureCanvasTkAgg(fig, master=third_frame)
canvas.draw()

waiting_label = tk.Label(third_frame, text="Waiting for data...", bg="white", font=("Helvetica", 16))
waiting_label.place(x=350, y=200)

waiting_label2 = tk.Label(third_frame, text="", bg="white", font=("Helvetica", 16))
waiting_label2.place(x=250, y=200)


# Use the place geometry manager to position the canvas
canvas.get_tk_widget().place(x=10, y=10, width=880, height=300)



root.mainloop()
